In [1]:
import pandas as pd
from experiments.models.GPTModel import GPTInference
from tqdm import tqdm
import re, string
import numpy as np
import math
from tqdm import tqdm 

In [2]:
df_llm = pd.read_csv('/home/sanjana/factual_evaluation_source_based/annotations/round1_aggredialfact_annotated/dialoguefact_two_annotators_round1.csv')
df_llm = df_llm[df_llm['user_id'] == 'ann_wjbp']
df_sota = pd.read_csv('/home/sanjana/factual_evaluation_source_based/datasets/sota_annotations/dialogue_finegrained_aggrefact.csv')
len(df_llm)



119

In [103]:
row_idx = 5
row = df_llm.iloc[[row_idx]]
summ = row['summary'].values[0]
dlg = row['dialogue'].values[0]
nonfactual_span = row['nonfactual_spans'].values[0]
summ

'Rashi, a student, expresses her confusion about choosing a career to her teacher. After a discussion, the teacher advises Rashi to follow her interests and passions, and not only focus on earning potential, while further suggesting that balance is key in any career. She suggests that Rashi should gain experience in her field of interest and then work to widen her scope with time, making it a blend of earning and learning. Rashi acknowledges the advice as helpful.'

In [104]:
nonfactual_span

'not only focus on earning potential'

In [121]:
import spacy 
nlp = spacy.load('en_core_web_sm')

def check_membership(token, compare_list):
    for each in compare_list:
        
        if token == each:
            return True
    return False 
    
def get_mask_infill(doc):
    all_masked_results = []
    mask_idx = []
    mask_tokens = []
    pos_tags = {
        'noun': ['PROPN', 'NOUN'],
        'verb': ['VERB'],
        'adjective': ['ADJ'],
        'adposition': ['ADP'],
        'pronoun': ['PRON'],
        'adverb': ['ADV']
                    
    }
    
    dep_tags = {
        'subjobj': ['nsubj', 'dobj', 'pobj'], 
        'negation': ['neg'],
        'preposition': ['prep'],
        
    }
    
    tags = {
        'verb': ['VBZ', 'VBG', 'VB'],
        
    }


    for idx, token in enumerate(doc):
        append = False
        # print(token.text, token.pos_, token.dep_ , token.tag_)
        # print(token.tag_, check_membership(token.tag_, tags['verb']))
        
        if check_membership(token.dep_, dep_tags['subjobj']) and check_membership(token.pos_, pos_tags['noun']):
            # print('SUBJOBJ')
            # print(token.text, token.pos_, token.dep_ , token.tag_)
            append = True
            token_type = 'subjobj'
            # print(token.text, token.pos_, token.dep_ , token.tag_)
    
        elif check_membership(token.pos_, pos_tags['adjective']):
            append = True
            token_type = 'attr'
            # print(token.text, token.pos_, token.dep_ , token.tag_)
            
        elif check_membership(token.tag_, tags['verb']) or check_membership(token.pos_, pos_tags['verb']):
            append = True
            token_type = 'pred'
            # print(token.text, token.pos_, token.dep_ , token.tag_)
    
        elif check_membership(token.dep_, dep_tags['negation']):
            append = True
            token_type = 'neg'
            # print(token.text, token.pos_, token.dep_ , token.tag_)
            
        elif check_membership(token.pos_, pos_tags['pronoun']):
            append = True
            token_type = 'coref'
            # print(token.text, token.pos_, token.dep_ , token.tag_)
            
        elif check_membership(token.dep_, dep_tags['preposition']) or \
        check_membership(token.pos_, pos_tags['adverb']) :
            append = True
            token_type = 'circ'
        
        if append:
                mask_idx.append(idx)
                mask_tokens.append((idx, token.text, token_type))

    error_types_indices_map = {}
    for token_idx, token_text, err_type in mask_tokens:
            if err_type not in error_types_indices_map:
                error_types_indices_map[err_type] = []
            error_types_indices_map[err_type] += [token_idx]
        
    for err_type, err_indices in error_types_indices_map.items():
            for idx in err_indices:
                masked_text = [each.text for each in origin_tokens[:idx]] + ['<BLANK>'] + [each.text for each in origin_tokens[idx+1:]]
                masked_text = ' '.join(masked_text)
                all_masked_results.append((masked_text, origin_tokens[idx].text, err_type))
    return all_masked_results


summ_doc =nlp(summ)
summ_sent_mask_infill = []
for summ_sent in summ_doc.sents:
    print(summ_sent)
    summ_sent = summ_sent.text
    doc = nlp(summ_sent)
    origin_tokens = [token for token in doc]
    summ_sent_mask_infill.append(get_mask_infill(doc))

Rashi, a student, expresses her confusion about choosing a career to her teacher.
After a discussion, the teacher advises Rashi to follow her interests and passions, and not only focus on earning potential, while further suggesting that balance is key in any career.
She suggests that Rashi should gain experience in her field of interest and then work to widen her scope with time, making it a blend of earning and learning.
Rashi acknowledges the advice as helpful.


In [122]:
summ_sent_mask_infill[1]

[('<BLANK> a discussion , the teacher advises Rashi to follow her interests and passions , and not only focus on earning potential , while further suggesting that balance is key in any career .',
  'After',
  'circ'),
 ('After a discussion , the teacher advises Rashi to follow her interests and passions , and not <BLANK> focus on earning potential , while further suggesting that balance is key in any career .',
  'only',
  'circ'),
 ('After a discussion , the teacher advises Rashi to follow her interests and passions , and not only focus <BLANK> earning potential , while further suggesting that balance is key in any career .',
  'on',
  'circ'),
 ('After a discussion , the teacher advises Rashi to follow her interests and passions , and not only focus on earning potential , while <BLANK> suggesting that balance is key in any career .',
  'further',
  'circ'),
 ('After a discussion , the teacher advises Rashi to follow her interests and passions , and not only focus on earning potential

In [144]:
import nltk
import re
import string 

def get_atomic_facts_gpt( text, text_type):
    instr = f'Write the {text_type} into indirect speech without introducing any unsupported information or inferences'
    prompt = f'{instr}\nDialogue: {text}'
    # print(prompt)
    gpt_response = GPTInference().get_chatgpt_response(prompt)
    # print(nltk.sent_tokenize(gpt_response))
    return gpt_response
    
def get_atomic_facts( dlg):
        atomic_facts = get_atomic_facts_gpt( dlg, 'dialogue')
        atomic_facts = nltk.sent_tokenize(atomic_facts)
        atomic_facts = [re.sub('[1-9]', '', each) for each in atomic_facts]
        atomic_facts = [each.strip(string.punctuation).strip() for each in atomic_facts]
        return atomic_facts
    
def get_nonfactual_comparators(all_masked_results):
    atomic_facts = ' '.join(get_atomic_facts(dlg))
    instruction = '''
    List the most appropriate answer that can fill in the <BLANK> in the sentence by extracting from the source. Ensure the sentence is consistent with source.
    If there are no appropriate answers reply "None"'''
    answers = []
    nonfactual_sent_span = []
    for sentence, answer, err_type in all_masked_results:   
        prompt = f'{instruction}\nSource: {atomic_facts}\nSentence: {sentence}'
        response = GPTInference().get_chatgpt_response(prompt)
        
        if "None" in response:
            response = [response]
        elif len(response.split('\n')) > 1:
            response = response.split('\n')
        elif len(response.split(',')) > 1:
            response = response.split(',')
        else:
            response = [response]
        response = [re.sub('[1-9]', '', each) for each in response]
        response = [each.strip(string.punctuation).strip() for each in response]
        
        if not check_membership(answer, response):
            print(sentence, answer, response)
            print('===')
        answers += [response]
    return answers

In [145]:
for mask_infill in [summ_sent_mask_infill[1]]:
    get_nonfactual_comparators(mask_infill)

After a discussion , the teacher advises Rashi to follow her interests and passions , and not <BLANK> focus on earning potential , while further suggesting that balance is key in any career . only ['solely']
===
After a discussion , the teacher advises Rashi to follow her interests and passions , and not only focus <BLANK> earning potential , while further suggesting that balance is key in any career . on ['on her']
===
After a discussion , the teacher advises Rashi to follow her interests and passions , and not only focus on earning potential , while <BLANK> suggesting that balance is key in any career . further ['also']
===
After a discussion , the teacher advises Rashi to follow her interests and passions , and not only focus on earning potential , while further suggesting that balance is key <BLANK> any career . in ['in choosing']
===
After a <BLANK> , the teacher advises Rashi to follow her interests and passions , and not only focus on earning potential , while further suggesting

In [101]:
# all_masked_results

In [138]:
atomic_facts = ' '.join(get_atomic_facts(dlg))

In [139]:
atomic_facts


'The teacher noticed that Rashi seemed to be a bit down and asked her why Rashi admitted that she was feeling confused about her career When the teacher asked for clarification, Rashi explained that she\'d been discussing career options with her friends, but there were too many options to choose from Encouragingly, the teacher advised Rashi to base her career choice on what truly interested her Rashi confessed that she had multiple interests and didn\'t understand how they would determine her career The teacher explained that it was Rashi\'s passion that would lead her to success When Rashi questioned the factor of earnings, the teacher reminded her that at some point, everyone must learn to balance their duties and success Wondering how to achieve that, Rashi questioned further The teacher advised her to choose a career that interested her, to gain experience in that field and try to progress and expand her scope over time Arguing that it was like an "earn and learn" sort of thing, th

In [51]:
row

,Unnamed: 0,uuid,user_id,docid,model,nonfactual_spans,evidence,error_type,summary,dialogue
9,9,02f228f5-435e-4906-bb98-1452bd51d708,ann_wjbp,13611508,gpt4-32k-0613,the products,I'll check them,Extrinsic_Error,"Martha, seeing Ophelia''s profile picture, is ...","Martha: Hey, can I ask you a question?\r\nOphe..."


In [ ]:
import torch
from transformers import AutoTokenizer, FalconForCausalLM

tokenizer = AutoTokenizer.from_pretrained("tiiuae/falcon-7b")
model = FalconForCausalLM.from_pretrained("tiiuae/falcon-7b")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model.generate(**inputs)


In [14]:
tokenizer.batch_decode(outputs)[0]


TypeError: argument 'ids': Can't extract `str` to `Vec`